In [ ]:
import os
import logging
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Any, Union, Tuple, Optional
import yaml
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix

sys.path.append(str(Path(__file__).resolve().parents[1] / "features"))
from text_processor import TextProcessor
# Import du TextProcessor
# Option 1: Si text_processor.py est dans src/features


# Option 2: Si text_processor.py est dans le même dossier que random_forest_model.py
# import sys
# from pathlib import Path
# sys.path.append(str(Path(__file__).parent))
# from text_processor import TextProcessor

# Configure logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

class RandomForestModel:
    """
    Classe pour l'entraînement, l'évaluation et l'utilisation d'un modèle RandomForest
    pour la classification de textes.
    """
    def __init__(self, config_path: Union[str, Path] = None):
        # Determine project root and default config path
        project_root = Path(__file__).resolve().parents[2]
        default_cfg = project_root / "config" / "config.yaml"
        self.config_path = Path(config_path) if config_path else default_cfg
        
        # Load config
        self.config = self._load_config(self.config_path)
        
        # Paths & params
        self.input_file = project_root / self.config.get("processed_file", "data/processed/x_train_clean.csv")
        self.model_output = project_root / self.config.get("model_output", "models/randomforest.pkl")
        
        # Model settings
        model_cfg = self.config.get("model", {})
        self.test_size = model_cfg.get("test_size", 0.2)
        self.random_state = model_cfg.get("random_state", 42)
        
        # RandomForest parameters
        rf_params = model_cfg.get("random_forest", {})
        self.n_estimators = rf_params.get("n_estimators", 100)
        self.max_depth = rf_params.get("max_depth", None)
        self.min_samples_split = rf_params.get("min_samples_split", 2)
        self.class_weight = rf_params.get("class_weight", "balanced")
        
        # Initialize model
        self.model = None
        self.text_processor = None
        self.classes_ = None

    def _load_config(self, path: Path) -> Dict[str, Any]:
        """Charge la configuration depuis un fichier YAML."""
        try:
            with open(path, "r", encoding="utf-8") as f:
                cfg = yaml.safe_load(f)
                logger.info(f"Configuration chargée depuis {path}")
                return cfg
        except Exception as e:
            logger.warning(f"Impossible de charger {path}: {e}\nUtilisation des valeurs par défaut.")
            return {}
    
    def _load_data(self) -> pd.DataFrame:
        """Charge les données prétraitées depuis le fichier CSV."""
        logger.info(f"Chargement des données depuis {self.input_file}")
        return pd.read_csv(self.input_file, encoding="utf-8")
    
    def _prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Prépare les données pour l'entraînement en utilisant le TextProcessor
        pour la vectorisation TF-IDF.
        """
        logger.info("Préparation des données pour l'entraînement...")
        
        X = data["text"].values
        y = data["label"].values
        self.classes_ = np.unique(y)
        
        # Split des données
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        # Vectorisation des textes avec le TextProcessor
        logger.info("Initialisation et entraînement du TextProcessor...")
        self.text_processor = TextProcessor(self.config_path)
        X_train_vec = self.text_processor.fit_transform(X_train)
        X_test_vec = self.text_processor.transform(X_test)
        
        logger.info(f"Données divisées: Train={X_train_vec.shape[0]}, Test={X_test_vec.shape[0]}")
        logger.info(f"Dimension des features: {X_train_vec.shape[1]}")
        
        return X_train_vec, X_test_vec, y_train, y_test
    
    def train(self, X_train: Optional[np.ndarray] = None, y_train: Optional[np.ndarray] = None) -> 'RandomForestModel':
        if X_train is None or y_train is None:
            data = self._load_data()
            X_train_vec, X_test_vec, y_train, y_test = self._prepare_data(data)
            # Si on a juste appelé train() sans arguments, on évalue aussi le modèle
            self._train_model(X_train_vec, y_train)
            self._evaluate_model(X_test_vec, y_test)
        else:
            # Si l'utilisateur a fourni explicitement X_train et y_train
            self._train_model(X_train, y_train)
        
        return self
    
    def _train_model(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
    
        logger.info(f"Entraînement du modèle RandomForest avec {X_train.shape[0]} exemples")
        logger.info(f"Paramètres: n_estimators={self.n_estimators}, max_depth={self.max_depth}")
        
        self.model = RandomForestClassifier(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            class_weight=self.class_weight,
            random_state=self.random_state,
            n_jobs=-1  # Utiliser tous les cœurs disponibles
        )
        
        self.model.fit(X_train, y_train)
        logger.info("Entraînement du modèle terminé")
    
    def evaluate(self, X_test: np.ndarray, y_test: np.ndarray) -> Dict[str, float]:

        return self._evaluate_model(X_test, y_test)
    
    def _evaluate_model(self, X_test: np.ndarray, y_test: np.ndarray) -> Dict[str, float]:
      
        if self.model is None:
            raise ValueError("Le modèle n'a pas été entraîné. Appelez d'abord train().")
        
        logger.info(f"Évaluation du modèle sur {X_test.shape[0]} exemples...")
        
        y_pred = self.model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        logger.info(f"Accuracy: {accuracy:.4f}")
        logger.info(f"F1 Score (weighted): {f1:.4f}")
        logger.info("\nRapport de classification détaillé:")
        logger.info(classification_report(y_test, y_pred))
        
        # Confusion matrix
        logger.info("\nMatrice de confusion:")
        cm = confusion_matrix(y_test, y_pred)
        logger.info(f"\n{cm}")
        
        return {
            'accuracy': accuracy,
            'f1_score': f1,
            'classification_report': classification_report(y_test, y_pred, output_dict=True)
        }
    
    def predict(self, texts: Union[str, list, np.ndarray]) -> np.ndarray:
        """
        Prédit la classe pour un ou plusieurs textes.
        
        Args:
            texts: Texte unique ou liste/array de textes à classifier
            
        Returns:
            np.ndarray: Prédictions des classes
        """
        if self.model is None or self.text_processor is None:
            raise ValueError("Le modèle ou le processeur de texte n'a pas été initialisé. Appelez d'abord train() ou load().")
        
        # Convert single text to array
        if isinstance(texts, str):
            texts = np.array([texts])
        elif isinstance(texts, list):
            texts = np.array(texts)
        
        # Transform text to TF-IDF features
        X = self.text_processor.transform(texts)
        
        # Make predictions
        return self.model.predict(X)
    
    def predict_proba(self, texts: Union[str, list, np.ndarray]) -> np.ndarray:
      
        if self.model is None or self.text_processor is None:
            raise ValueError("Le modèle ou le processeur de texte n'a pas été initialisé. Appelez d'abord train() ou load().")
        
        # Convert single text to array
        if isinstance(texts, str):
            texts = np.array([texts])
        elif isinstance(texts, list):
            texts = np.array(texts)
        
        # Transform text to TF-IDF features
        X = self.text_processor.transform(texts)
        
        # Make probabilistic predictions
        return self.model.predict_proba(X)
    
    def save(self, output_path: Union[str, Path] = None) -> None:
        """
        Sauvegarde le modèle et le processeur de texte dans un fichier.
        
        Args:
            output_path: Chemin où sauvegarder le modèle (facultatif)
        """
        if self.model is None or self.text_processor is None:
            raise ValueError("Le modèle ou le processeur de texte n'a pas été initialisé. Appelez d'abord train() ou load().")
        
        save_path = Path(output_path) if output_path else self.model_output
        save_path.parent.mkdir(parents=True, exist_ok=True)
        
        logger.info(f"Sauvegarde du modèle et du processeur de texte dans {save_path}")
        
        # Save both model and text processor
        joblib.dump({
            'model': self.model,
            'text_processor_vectorizer': self.text_processor.vectorizer,
            'classes_': self.classes_
        }, save_path)
        
        logger.info(f"Modèle sauvegardé avec succès dans {save_path}")
    
    def load(self, input_path: Union[str, Path] = None) -> 'RandomForestModel':
        """
        Charge un modèle préentraîné et son processeur de texte depuis un fichier.
        
        Args:
            input_path: Chemin du fichier modèle à charger (facultatif)
            
        Returns:
            self: Retourne l'instance de la classe pour permettre le chaînage des méthodes
        """
        load_path = Path(input_path) if input_path else self.model_output
        
        logger.info(f"Chargement du modèle depuis {load_path}")
        
        # Load model and vectorizer
        saved_data = joblib.load(load_path)
        
        self.model = saved_data['model']
        self.classes_ = saved_data.get('classes_', self.model.classes_)
        
        # Create and initialize text processor
        self.text_processor = TextProcessor(self.config_path)
        self.text_processor.vectorizer = saved_data['text_processor_vectorizer']
        
        logger.info(f"Modèle chargé avec succès: {self.model}")
        return self

def train_and_save_model(config_path: Union[str, Path] = None) -> None:
    """
    Fonction utilitaire pour entraîner et sauvegarder un modèle RandomForest.
    
    Args:
        config_path: Chemin du fichier de configuration (facultatif)
    """
    logger.info("Démarrage de l'entraînement du modèle RandomForest...")
    
    try:
        # Créer et entraîner le modèle
        rf_model = RandomForestModel(config_path)
        rf_model.train()  # Charge les données, entraîne et évalue le modèle
        
        # Sauvegarder le modèle et le vectoriseur
        rf_model.save()
        
        logger.info("Processus d'entraînement terminé avec succès!")
        
    except Exception as e:
        logger.error(f"Erreur durant l'entraînement du modèle: {e}")
        raise


if __name__ == "__main__":
    # Exécuter l'entraînement du modèle si le script est lancé directement
    train_and_save_model()

In [3]:
import joblib

model = joblib.load("../models/random_forest_model.pkl")
print(model)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
                ('clf', RandomForestClassifier(random_state=42))])
